In [1]:
import audible_plot as ap
import ccxt
import numpy as np
import pandas as pd
import pandas_ta as ta

binanceusdm = ccxt.binanceusdm()

In [2]:
candles = np.array(binanceusdm.fetch_ohlcv("BTCUSDT", "5m"))

In [32]:
data = pd.DataFrame(
    data=candles[:, 1:],
    index=pd.to_datetime(pd.Series(candles[:, 0], name="timestamp"), unit="ms"),
    columns=["open", "high", "low", "close", "volume"],
)

aroon = ta.aroon(data["high"], data["low"], 14)
macd = ta.macd(data["close"])
rsi = ta.rsi(data["close"], 14)
data = pd.concat(
    objs=[
        data,
        aroon,
        ta.tsignals(ta.above(macd["MACD_12_26_9"], macd["MACDs_12_26_9"])).add_prefix(
            "LONG_"
        ),
        ta.tsignals(ta.below(macd["MACD_12_26_9"], macd["MACDs_12_26_9"])).add_prefix(
            "SHORT_"
        ),
        rsi,
        macd,
    ],
    axis=1,
)

data.dropna(inplace=True)
data.to_clipboard()
data

,open,high,low,close,volume,AROOND_14,AROONU_14,AROONOSC_14,LONG_TS_Trends,LONG_TS_Trades,LONG_TS_Entries,LONG_TS_Exits,SHORT_TS_Trends,SHORT_TS_Trades,SHORT_TS_Entries,SHORT_TS_Exits,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
timestamp,,,,,,,,,,,,,,,,,,,,
2024-11-02 13:20:00,69602.0,69649.0,69600.8,69629.2,410.470,64.285714,100.000000,35.714286,1,1,1,0,0,0,0,0,57.954797,-16.989270,14.066010,-31.055280
2024-11-02 13:25:00,69628.7,69700.0,69628.4,69669.9,464.031,57.142857,100.000000,42.857143,1,0,0,0,0,0,0,0,61.660307,-6.254098,19.840945,-26.095043
2024-11-02 13:30:00,69670.0,69670.0,69573.8,69592.3,310.621,50.000000,92.857143,42.857143,1,0,0,0,0,0,0,0,52.212004,-3.962392,17.706121,-21.668513
2024-11-02 13:35:00,69592.3,69597.0,69435.0,69536.1,986.268,42.857143,85.714286,42.857143,1,0,0,0,0,0,0,0,46.638229,-6.604932,12.050865,-18.655797
2024-11-02 13:40:00,69536.2,69579.8,69485.3,69578.5,327.517,35.714286,78.571429,42.857143,1,0,0,0,0,0,0,0,50.897153,-5.217692,10.750484,-15.968176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-04 03:50:00,69111.5,69154.8,69108.6,69134.9,181.505,71.428571,28.571429,-42.857143,0,0,0,0,1,0,0,0,53.749226,62.121277,-24.145102,86.266379
2024-11-04 03:55:00,69135.0,69150.0,68980.0,69034.0,939.263,100.000000,21.428571,-78.571429,0,0,0,0,1,0,0,0,46.851987,48.215675,-30.440564,78.656239
2024-11-04 04:00:00,69034.0,69117.5,69033.9,69095.3,477.551,92.857143,14.285714,-78.571429,0,0,0,0,1,0,0,0,50.968524,41.661534,-29.595763,71.257298


In [35]:
from datetime import timedelta


chart = ap.AudibleChart(
    data=data,
    config=[
        ap.SeriesConfig(
            key="close",
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(ap.ToneGenerator.WaveType.sawtooth),
                volume=0.1,
            ),
        ),
        ap.SeriesConfig(
            key="SHORT_TS_Trades",
            is_extra=True,
            renderer=ap.ConditionalRenderer(
                condition=lambda v: v == 1 or v == -1,
                renderer=ap.PitchDataRenderer(
                    enable_transitions=False,
                    generator=ap.ToneGenerator(),
                    pan=-0.5,
                ),
            ),
            range=ap.FixedRange(-1, 1),
        ),
        ap.SeriesConfig(
            key="LONG_TS_Trades",
            is_extra=True,
            range=ap.FixedRange(-1, 1),
            renderer=ap.ConditionalRenderer(
                condition=lambda v: v == 1 or v == -1,
                renderer=ap.PitchDataRenderer(
                    enable_transitions=False,
                    generator=ap.ToneGenerator(),
                    pan=0.5,
                ),
            ),
        ),
        ap.SeriesConfig(
            key="RSI_14",
            is_extra=True,
            range=ap.FixedRange(0, 100),
            renderer=ap.ConditionalRenderer(
                condition=lambda v: v < 30 or v > 70 - 1,
                renderer=ap.PitchDataRenderer(
                    enable_transitions=False,
                    generator=ap.ToneGenerator(),
                    pan=0.2,
                ),
            ),
        ),
        ap.SeriesConfig(
            key="MACD_12_26_9",
            is_extra=True,
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(),
                pan=-0.2,
            ),
        ),
        ap.SeriesConfig(
            key="MACDs_12_26_9",
            is_extra=True,
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(),
                pan=0.2,
            ),
        ),
    ],
    frequency_range=ap.FixedRange(300, 800),
    sample_rate=44100,
)

window = chart.window(slice(-400, None))
window.play(
    [
        "LONG_TS_Trades",
        "SHORT_TS_Trades",
        "close",
        # "MACDs_12_26_9",
        # "MACD_12_26_9",
    ],
    duration=timedelta(milliseconds=200),
)

,open,high,low,close,volume,AO_5_34,APO_12_26,BIAS_SMA_26,BOP,AR_26,BR_26,CCI_14_0.015,CFO_9,CG_10,CMO_14,COPC_11_14_10,CTI_12,ER_10,BULLP_13,BEARP_13,FISHERT_9_1,FISHERTs_9_1,INERTIA_20_14,K_9_3,D_9_3,J_9_3,KST_10_15_20_30_10_10_10_15,KSTs_9,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MOM_10,PGO_14,PPO_12_26_9,PPOh_12_26_9,PPOs_12_26_9,PSL_12,PVO_12_26_9,PVOh_12_26_9,PVOs_12_26_9,QQE_14_5_4.236,QQE_14_5_4.236_RSIMA,QQEl_14_5_4.236,QQEs_14_5_4.236,ROC_10,RSI_14,RSX_14,RVGI_14_4,RVGIs_14_4,SLOPE_1,SMI_5_20_5,SMIs_5_20_5,SMIo_5_20_5,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO,SQZPRO_20_2.0_20_2_1.5_1,SQZPRO_ON_WIDE,SQZPRO_ON_NORMAL,SQZPRO_ON_NARROW,SQZPRO_OFF,SQZPRO_NO,STC_10_12_26_0.5,STCmacd_10_12_26_0.5,STCstoch_10_12_26_0.5,STOCHk_14_3_3,STOCHd_14_3_3,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,TRIX_30_9,TRIXs_30_9,TSI_13_25_13,TSIs_13_25_13,UO_7_14_28,WILLR_14
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-11-02 10:35:00,69496.7,69566.0,69492.0,69558.9,300.829,NaN,NaN,NaN,0.840541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0,0,0,1,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-02 10:40:00,69558.9,69581.8,69555.5,69569.0,199.457,NaN,NaN,NaN,0.384030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.1,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0,0,0,1,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-02 10:45:00,69569.0,69591.9,69543.7,69574.1,163.729,NaN,NaN,NaN,0.105809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0,0,0,1,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-02 10:50:00,69574.0,69609.6,69570.8,69609.6,209.077,NaN,NaN,NaN,0.917526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.5,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0,0,0,1,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-02 10:55:00,69609.6,69632.9,69604.5,69623.6,132.674,NaN,NaN,NaN,0.492958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0,0,0,1,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-04 03:50:00,69111.5,69154.8,69108.6,69134.9,181.505,111.432059,66.093590,0.000254,0.506494,130.149985,130.149985,-83.102443,0.002999,-5.500817,7.498452,0.018613,-0.786586,0.391789,6.514851,-39.685149,-1.530381,-1.556221,52.095264,25.606459,26.124245,24.570887,501.507294,614.425406,62.121277,-24.145102,86.266379,-175.6,-0.416528,0.095625,-0.078106,0.173731,41.666667,-18.005302,-10.856139,-7.149163,60.850686,54.863281,NaN,60.850686,-0.253353,53.749226,45.216992,-0.024133,0.027571,23.4,0.062657,0.110167,-0.047510,-39.600000,1,0,0,-39.600000,1,1,0,0,0,0.177641,62.121277,0.445943,14.208503,20.412019,3.040416,8.565638,0.010136,0.008636,15.600931,20.024259,49.471066,-85.687114
2024-11-04 03:55:00,69135.0,69150.0,68980.0,69034.0,939.263,89.127059,39.657692,-0.001282,-0.594118,113.034463,113.049696,-155.526996,-0.086160,-5.501272,-6.296025,-0.135051,-0.790098,0.387278,18.041301,-151.958699,-1.753069,-1.530381,48.745084,24.020780,25.423090,21.216159,459.345706,591.122575,48.215675,-30.440564,78.656239,-172.3,-